# Modeling

In [7]:
from operator import mod
from os import getcwd
from os.path import exists, join
import os

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import random
import ast


In [8]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings
from sklearn.metrics import classification_report
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer

from sklearn.base import TransformerMixin

In [9]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

from tqdm import tqdm

tqdm.pandas()

In [10]:
# Depression screener
dep_screener_cols = [
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks'
]
model_features_opt2 = dep_screener_cols + [
    'seen_mental_health_professional',
    'times_with_12plus_alc',
    'time_since_last_healthcare',
    'cholesterol_prescription',
    'high_cholesterol',
    'age_in_years',
    'horomones_not_bc',
    'months_since_birth',
    'arthritis',
    'high_bp',
    'regular_periods',
    'moderate_recreation',
    'thyroid_issues',
    'vigorous_recreation',
    'stroke',
    'is_usa_born',
    'asthma',
    'count_days_moderate_recreational_activity',
    'have_health_insurance',
    'num_dep_screener_0',
    'weight_lbs_over_height_in_ratio'
]

model_features_low_opt7 = [
    'count_days_seen_doctor_12mo_bin',
    'times_with_12plus_alc',
    'seen_mental_health_professional',
    'count_lost_10plus_pounds',
    'arthritis',
    'horomones_not_bc',
    'is_usa_born',
    'times_with_8plus_alc',
    'time_since_last_healthcare',
    'duration_last_healthcare_visit',
    'work_schedule'
]

columns_to_use_low = model_features_low_opt7
columns_to_use_high = model_features_opt2

In [43]:
len(list(set(columns_to_use_low + columns_to_use_high)))

36

In [23]:
# create bins using estimator
dirname = os.path.dirname('./')
est = joblib.load(os.path.join(dirname, "model_kbins.pkl"))

clf_low = joblib.load(os.path.join(dirname, "model_pipeline_low.pkl"))
clf_high = joblib.load(os.path.join(dirname, "model_pipeline_high.pkl"))



In [33]:
ql = '''# RIDAGEYR: age_in_years
            "question": "How old are you? (Please select the closest option)",
            "options": list(range(32, 33)),
            "option_is_continuous": True,
            "var_code": "age_in_years",
        },
        {
            # height: height_in for weight_lbs_over_height_in_ratio
            "question": "What is your height in inches? (Please select the closest option)",
            "options": list(range(62, 63)),
            "option_is_continuous": True,
            "var_code": "height_in",
        },
        {
            # weight: weight_lbs for weight_lbs_over_height_in_ratio
            "question": "What is your weight in pounds? (Please select the closest option)",
            "options": list(range(155, 156)),
            "option_is_continuous": True,
            "var_code": "weight_lbs",
        },
        {
            # DMDBORN4: is_usa_born
            "question": "Were you born in the United States?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "is_usa_born",
        },
        {
            # HIQ011: have_health_insurance
            "question": "Are you covered by health insurance or some kind of health care plan?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "have_health_insurance",
        },
        {
            # RHQ197: months_since_birth
            "question": "How many months ago did you have your baby? (Please select the closest option)",
            "options": list(range(3, 4)),
            "option_is_continuous": True,
            "var_code": "months_since_birth",
        },
        {
            # RHQ031: regular_periods
            "question": "Have you had at least one menstrual period in the past 12 months?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "regular_periods",
        },
        {
            # RHQ540: horomones_not_bc
            "question": "Have you ever used female hormones such as estrogen and progesterone? Please include any forms such as pills, cream, patch, and inejctables, but do not include birth controls methods or use for infertility.",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "horomones_not_bc",
        },
        {
            # HUD062: time_since_last_healthcare
            "question": "About how long has it been since you last saw a health care professional about your health for any reason?",
            "options": [
                "Dummy",
                "Never",
                "Within the past year",
                "Within the last 2 years",
                "Within the last 5 years",
                "5 years ago or more",
            ],
            "option_codes": [None, 0, 1, 2, 3, 4],
            "var_code": "time_since_last_healthcare",
        },
        {
            # HUQ090: seen_mental_health_professional
            "question": "During the past 12 months, have you seen or talked to a mental health professional about your health?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "seen_mental_health_professional",
        },
        {
            # DPQ010: little_interest_in_doing_things
            "question": "Over the last 2 weeks, how often have you been bothered by the following problems: little interest or pleasure in doing things?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "little_interest_in_doing_things",
        },
        {
            # DPQ020: feeling_down_depressed_hopeless
            "question": "Over the last 2 weeks, how often have you been bothered by feeling down, depressed, or hopeless?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "feeling_down_depressed_hopeless",
        },
        {
            # DPQ030: trouble_falling_or_staying_asleep
            "question": "Over the last 2 weeks, how often have you been bothered by trouble falling or staying asleep, or sleeping too much?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "trouble_falling_or_staying_asleep",
        },
        {
            # DPQ040: feeling_tired_or_having_little_energy
            "question": "Over the last 2 weeks, how often have you been bothered by feeling tired or having little energy?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "feeling_tired_or_having_little_energy",
        },
        {
            # DPQ050: poor_appetitie_or_overeating
            "question": "Over the last 2 weeks, how often have you been bothered by having poor appetite or overeating?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "poor_appetitie_or_overeating",
        },
        {
            # DPQ060: feeling_bad_about_yourself
            "question": "Over the last 2 weeks, how often have you been bothered by feeling bad about yourself - or that you are a failure or have let yourself or your family down?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "feeling_bad_about_yourself",
        },
        {
            # DPQ070: trouble_concentrating
            "question": "Over the last 2 weeks, how often have you been bothered by trouble concentrating on things, such as reading the newspaper or watching TV?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "trouble_concentrating",
        },
        {
            # DPQ080: moving_or_speaking_to_slowly_or_fast
            "question": "Over the last 2 weeks, how often have you been bothered by moving or speaking so slowly that other people could have noticed? Or the opposite - being so fidgety or restless that you have been moving around a lot more than usual?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "moving_or_speaking_to_slowly_or_fast",
        },
        {
            # DPQ090: thoughts_you_would_be_better_off_dead
            "question": "Over the last 2 weeks, how often have you been bothered by thoughts that you would be better off dead or of hurting yourself in some way?",
            "options": [
                "Dummy",
                "Not at all",
                "Several days",
                "More than half the days",
                "Nearly every day",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "thoughts_you_would_be_better_off_dead",
        },
        {
            # DPQ100: difficult_doing_daytoday_tasks
            "question": "How difficult have these problems made it for you to do your work, take care of things at home, or get along with people?",
            "options": [
                "Dummy",
                "Not at all difficult",
                "Somewhat difficult",
                "Very difficult",
                "Extremely difficult",
            ],
            "option_codes": [None, 0, 1, 2, 3],
            "var_code": "difficult_doing_daytoday_tasks",
        },
        {
            # ALQ290: times_with_12plus_alc
            "question": "During the past 12 months, about how often did you have 12 or more drinks in a single day?",
            "options": [
                "Dummy",
                "Never in the last year",
                "Every day",
                "Nearly every day",
                "3-4 times a week",
                "2 times a week",
                "Once a week",
                "2-3 times a month",
                "Once a month",
                "7-11 times in the last year",
                "3-6 times in the last year",
                "1-2 times in the last year",
            ],
            "option_codes": [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            "var_code": "times_with_12plus_alc",
        },
        {
            # BPQ080: high_cholesterol
            "question": "Have you ever been told by a health care professional that your blood cholesterol level was high?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "high_cholesterol",
        },
        {
            # BPQ090D: cholesterol_prescription
            "question": "Have you ever been told by a health care professional to take prescribed medicine to lower blood cholesterol levels?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "cholesterol_prescription",
        },
        {
            # BPQ020: high_bp
            "question": "Has a health professional ever told you that you have/had hypertension (high blood pressure)?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "high_bp",
        },
        {
            # PAQ665: moderate_recreation
            "question": "In a typical week, do you do any moderate intensity recreational activities that cause a small increase in breathing/heart rate such as brisk walking, bicycling, or swimming for at least 10 minutes continuously?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "moderate_recreation",
        },
        {
            # PAQ670: count_days_moderate_recreational_activity
            "question": "In a typical week, on how many days do you do moderate intensity recreational activities?",
            "options": ["Dummy", "1", "2", "3", "4", "5", "6", "7"],
            "option_codes": [None, 1, 2, 3, 4, 5, 6, 7],
            "var_code": "count_days_moderate_recreational_activity",
        },
        {
            # PAQ650: vigorous_recreation
            "question": "In a typical week, do you do any vigorous intensity recreational activities that cause large increases in breathing/heart rate like running or basketball for at least 10 minutes continuously?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "vigorous_recreation",
        },
        {
            # MCQ160m: thyroid_issues
            "question": "Has a health professional ever told you that you had a thyroid problem?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "thyroid_issues",
        },
        {
            # MCQ160a: arthritis
            "question": "Has a health professional ever told you that you have/had arthritis",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "arthritis",
        },
        {
            # MCQ160f: stroke
            "question": "Has a health professional ever told you that you had a stroke?",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "stroke",
        },
        {
            # MCQ010: asthma
            "question": "Has a health professional ever told you that you have asthma??",
            "options": ["Dummy", "Yes", "No"],
            "option_codes": [None, 1, 2],
            "var_code": "asthma",
        },
        {
            # WHQ225: count_lost_10plus_pounds
            "question": "How many times have you lost 10 pounds or more because you were trying to lose weight? ",
            "options": ["1 to 2", "3 to 5", "6 to 10", "11 times or more", "Never"],
            "option_codes": [1, 2, 3, 4, 5],
            "var_code": "count_lost_10plus_pounds",
        },
        {
            #  ALQ280: times_with_8plus_alc
            "question": "During the past 12 months, about how often did you have 8 or more drinks in a single day?",
            "options": [
                "Dummy",
                "Never in the last year",
                "Every day",
                "Nearly every day",
                "3-4 times a week",
                "2 times a week",
                "Once a week",
                "2-3 times a month",
                "Once a month",
                "7-11 times in the last year",
                "3-6 times in the last year",
                "1-2 times in the last year",
            ],
            "option_codes": [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            "var_code": "times_with_8plus_alc",
        },
        {
            # HUD062: duration_last_healthcare_visit
            "question": "About how long has it been since you last saw a doctor or other health care professional about your health for any reason?",
            "options": [
                "Never",
                "Within the past year (anytime less than 12 months ago)",
                "Within the last 2 years (1 year but less than 2 years ago)",
                "Within the last 5 years (2 years but less than 5 years ago)",
                "5 years ago or more",
            ],
            "option_codes": [0, 1, 2, 3, 4],
            "var_code": "duration_last_healthcare_visit",
        },
        {
            # OCQ670: work_schedule
            "question": "Which of the following best describes your overall work schedule (include all jobs) for the last three months?",
            "options": [
                "Traditional 9 AM to 5 PM day",
                "Evening or nights",'''.split('#')




In [42]:
len([col[:50] for col in ql if len(col) > 5])

35

In [46]:

data = {
       "little_interest_in_doing_things": 1.0,
            "feeling_down_depressed_hopeless": 1.0,
            "trouble_falling_or_staying_asleep": 0.0,
            "feeling_tired_or_having_little_energy": 0.0,
            "poor_appetitie_or_overeating": 0.0,
            "feeling_bad_about_yourself": 0.0,
            "trouble_concentrating": 0.0,
            "moving_or_speaking_to_slowly_or_fast": 0.0,
            "thoughts_you_would_be_better_off_dead": 0.0,
            "difficult_doing_daytoday_tasks": 0.0,
            "count_days_seen_doctor_12mo":7.3,
            "seen_mental_health_professional": 10.0,
            "times_with_12plus_alc": 11.0,
            "time_since_last_healthcare": 12.0,
            "cholesterol_prescription": 1.0,
            "high_cholesterol": 1.0,
            "age_in_years": 1.0,
            "horomones_not_bc": 16.0,
            "months_since_birth": 1.0,
            "arthritis": 18.0,
            "high_bp": 1.0,
            "regular_periods": 1.0,
            "moderate_recreation": 1.0,
            "thyroid_issues": 1.0,
            "vigorous_recreation": 1.0,
            "stroke": 1.0,
            "is_usa_born": 25.0,
            "asthma": 1.0,
            "count_days_moderate_recreational_activity": 1.0,
            "have_health_insurance": 10.0,
            "weight_lbs": 150.0,
            "height_in": 65.0,
            "count_lost_10plus_pounds": 31.0,
            "times_with_8plus_alc": 32.0,
            "duration_last_healthcare_visit": 33.0,
            "work_schedule": 34.0}


X = np.array([[np.nan if val is None else val for val in data.values()]])
# use low
if (X[0:, 0:11] == 0).sum() < 9:
    # subset to features for this model

    X_low = np.take(X, [11, 10, 31, 18, 16, 25, 32, 12, 33, 34], axis=1)
    if not np.isnan(X_low[0, 1]):
        # count_days_seen_doctor_12mo_bin
        # create bins using estimator
        print(X_low)
        feature_values = np.array([X_low[0, 1]]).reshape([-1, 1])
        print(feature_values)
        feature_values = est.transform(feature_values)
        print(feature_values)
        X_low = np.append([[np.nan]], X_low, axis=1)
    else:
        X_low = np.append([[np.nan]], X_low, axis=1)
    # impute and scale
    imputer_low = SimpleImputer(strategy="median")
    trans_low = RobustScaler()
    # X_low = imputer_low.fit_transform(X_low)
    # X_low = trans_low.fit_transform(X_low)
    X_low = np.nan_to_num(X_low)
    # logging.warning(f"low model: {X_low}")
    pred = clf_low.predict(X_low)

else:
    # num_dep_screener_0
    X_1 = np.append(X[:, :29], [[(X[0:, 0:11] == 0).sum()]], axis=1)
    # weight_lbs_over_height_in_ratio
    X_2 = np.append(X_1, [[(X[0, 29] / X[0, 30])]], axis=1)
    # impute and scale
    imputer_high = SimpleImputer(strategy="median")
    trans_high = RobustScaler()
    # X_high = imputer_high.fit_transform(X_2)
    # X_high = trans_high.fit_transform(X_high)
    # logging.warning(f"high model: {X_high}")
    X_high = np.nan_to_num(X_2)
    pred = clf_high.predict(X_high)

# pred = clf.predict(survey_features)
pred = np.reshape(pred, (-1, 1))
# pred_formatted = [Prediction(prediction=p) for p in pred]
# preds = Predictions(predictions=pred_formatted)
# print(preds)

print(pred)

[[10.   7.3 65.   1.   1.   1.  31.  11.  32.  33. ]]
[[7.3]]
[[3.]]
[[0]]


In [16]:
# v2 csv
df_cdc_clean = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = df_cdc_clean[df_cdc_clean['has_been_pregnant'] == 1]
print(cdc_survey_pmom.shape)

# add features
cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)
cdc_survey_pmom['weight_lbs_over_height_in_ratio'] = round(df_cdc_clean['weight_lbs'] / cdc_survey_pmom['height_in'],1)

feature_values = cdc_survey_pmom['count_days_seen_doctor_12mo'].dropna().values
feature_values = feature_values.reshape([feature_values.shape[0],1])



# est.fit(feature_values)

feature_values = est.transform(feature_values)

fill_arr = cdc_survey_pmom['count_days_seen_doctor_12mo'].values.copy()
fill_arr[~np.isnan(fill_arr)] = np.asarray([val[0] for val in feature_values])
cdc_survey_pmom['count_days_seen_doctor_12mo_bin'] = fill_arr





(7741, 863)


/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_35630/2580352050.py:2: DtypeWarning: Columns (664,665) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cdc_clean = pd.read_csv('../../data/cdc_nhanes_survey_responses_clean.csv')
/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_35630/2580352050.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_survey_pmom['num_dep_screener_0'] = (cdc_survey_pmom[dep_screener_cols]==0).sum(axis=1)
/var/folders/0k/kvbyl6wn7kl2qbt7_94pckjw0000gn/T/ipykernel_35630/2580352050.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [18]:
feature_values

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [5.],
       [0.],
       [1.],
       [1.],
       [5.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [1.],
       [2.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [2.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [2.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [2.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [1.],
       [1.],
       [0.],
       [2.],
       [5.],
       [1.],
       [3.],
       [2.],
       [0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [2.],
       [4.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],